In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from PIL import Image
import os

TRAIN_SET = 1
SCAN_DEPTH = 16

tr_dirname = 'train/'
tr_set_dirname = os.path.join(tr_dirname, str(TRAIN_SET))
tr_img_dirname = os.path.join(tr_set_dirname, 'surface_volume')
tr_images = sorted([os.path.join(tr_img_dirname, filename) for filename in os.listdir(tr_img_dirname) if filename[-3:]=='tif'][:SCAN_DEPTH])


In [2]:
mask = np.array(Image.open(os.path.join(tr_set_dirname, "mask.png")).convert('1'))
labels = np.array(Image.open(os.path.join(tr_set_dirname, "inklabels.png")).convert('1'))
images = np.array([np.array(Image.open(filename), dtype=np.float32)/65535.0 for filename in tr_images]).transpose()

In [3]:
import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data, mask, labels, px_offset=10, img_depth=4,
                 include_rect=None, exclude_rect=None,
                 batch_size=32, shuffle=True, name='', sample_count=0):
        'Initialization'
        self.px_offset = px_offset
        self.include_rect = include_rect
        self.exclude_rect = exclude_rect
        self.dim = (2*self.px_offset+1, 2*self.px_offset+1, img_depth)
        self.batch_size = batch_size
        self.labels = labels
        self.data = data
        self.mask = mask
        self.shuffle = shuffle
        self.name = name
        self.batches = dict()
        self.cnt = sample_count
        
        self.data_gen = self.__data_generator()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return self.cnt // self.batch_size

    def __getitem__(self, index):
        if index not in self.batches.keys():
            self.batches[index] = next(self.data_gen)
        return self.batches[index]

    def on_epoch_end(self):
        pass

    def __data_generator(self):
        # Go through all pixels, collect next batch and return it
        X = np.empty((self.batch_size, *self.dim, 1))
        print(X.shape)
        y = np.empty((self.batch_size), dtype=int)
        batch_cnt = 0
        cur_batch_cnt = 0
        for pixel in zip(*np.where(self.mask==1)):            
            if not DataGenerator.pixel_ok(pixel, self.px_offset, self.mask):
                continue # Too close to the edge
            if self.include_rect:
                if DataGenerator.pixel_inside(pixel, self.include_rect):
                    X[cur_batch_cnt, :, :, :, 0] = self.get_large_pixel(pixel)
                    y[cur_batch_cnt] = self.labels[pixel[0],pixel[1]]
                    cur_batch_cnt += 1
            elif self.exclude_rect:
                if not DataGenerator.pixel_inside(pixel, self.exclude_rect):
                    X[cur_batch_cnt, :, :, :, 0] = self.get_large_pixel(pixel)
                    y[cur_batch_cnt] = self.labels[pixel[0],pixel[1]]
                    cur_batch_cnt += 1
            else:
                X[cur_batch_cnt, :, :, :, 0] = self.get_large_pixel(pixel)
                y[cur_batch_cnt] = self.labels[pixel[0],pixel[1]]
                cur_batch_cnt += 1
            if cur_batch_cnt == self.batch_size:
                print(f'returning batch #{batch_cnt}')
                yield X, y.reshape((-1,1))
                X = np.empty((self.batch_size, *self.dim, 1))
                y = np.empty((self.batch_size), dtype=int)
                cur_batch_cnt = 0
                batch_cnt += 1                
    
    #staticmethod
    def pixel_ok(pixel, offset, mask):
        return (offset <= pixel[1] < mask.shape[1] - offset) and (offset <= pixel[0] < mask.shape[0] - offset)

    #staticmethod
    def pixel_inside(pixel, rect):
        return (rect[0] <= pixel[1] <= rect[0] + rect[2]) and (rect[1] <= pixel[0] <= rect[1] + rect[3])
    
    def get_large_pixel(self, pixel):
        return self.data[
            pixel[0]-self.px_offset:pixel[0]+self.px_offset+1,
            pixel[1]-self.px_offset:pixel[1]+self.px_offset+1,
            :
        ]

In [4]:
PIXEL_SIZE = 16
BATCH_SIZE = 512

VAL_RECT = (2064, 2864, 1000, 1000)
# Determine the number of samples
train_sample_cnt = 0
val_sample_cnt = 0
px_offset = PIXEL_SIZE // 2
for c, pixel in enumerate(zip(*np.where(mask==1))):
    if c % 3000000 == 0:
        print(f'#{c}')
    if not DataGenerator.pixel_ok(pixel, px_offset, mask):
        continue # Too close to the edge
    if DataGenerator.pixel_inside(pixel, VAL_RECT):
        val_sample_cnt += 1
    else:
        train_sample_cnt += 1

# Generators
training_generator = DataGenerator(images, mask, labels, px_offset=px_offset,
                                   img_depth=SCAN_DEPTH, exclude_rect=VAL_RECT,
                                   batch_size=BATCH_SIZE, shuffle=True, name='TRAIN',
                                   sample_count=train_sample_cnt)
validation_generator = DataGenerator(images, mask, labels, px_offset=px_offset,
                                     img_depth=SCAN_DEPTH, include_rect=VAL_RECT,
                                     batch_size=BATCH_SIZE, shuffle=True, name='VAL',
                                     sample_count=val_sample_cnt)

#0
#3000000
#6000000
#9000000
#12000000
#15000000
#18000000
#21000000
#24000000
#27000000


In [5]:
from keras import Sequential
from keras.layers import Conv3D, MaxPooling3D, Flatten, Dense
from tensorflow.keras.optimizers.legacy import Adam

real_px_size = 2*(PIXEL_SIZE//2) + 1
input_shape = (real_px_size, real_px_size, SCAN_DEPTH, 1)
print(input_shape)
model = Sequential()
model.add(Conv3D(16, kernel_size=(4, 4, 4), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
model.add(MaxPooling3D(pool_size=(2, 2, 1)))
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
# model.add(MaxPooling3D(pool_size=(2, 2, 1)))
model.add(Conv3D(16, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
# model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])


(17, 17, 16, 1)
Metal device set to: Apple M2


2023-03-21 13:15:04.890583: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-21 13:15:04.890993: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
# Train model on dataset
model.fit(x=training_generator,
          validation_data=validation_generator,
          epochs=4,
          use_multiprocessing=True,
#           workers=3,
          shuffle=False,
          verbose=2)